# Crypto Portfolio & Top 20 Tracker

Bu proje, Python + Flask kullanarak geliştirilen basit ama işlevsel bir web uygulamasıdır.

Amacı:

Binance Top API’sinden market cap’e göre en popüler 20 kripto parayı çekmek

Kullanıcıya bu coinlerin anlık fiyatlarını göstermek

Kullanıcının kendi portföyüne coin eklemesine / çıkarmasına izin vermek

Portföydeki coinlerin toplam kâr / zarar durumunu hesaplamak

Portföyün kalıcı olması için JSON dosyasına kaydedilmesi

In [1]:
# Adım 1: Gerekli kütüphaneler ve Flask ayarları
from flask import Flask, request, render_template_string, redirect, url_for
from datetime import datetime
import requests, os, json   # API çağrısı + dosya işlemleri

# Flask uygulamasını başlat
app = Flask(__name__)
app.secret_key = "dev-secret-change-me"   # Prod ortamında değiştirilmeli

# Portföy dosyası (JSON formatında)
PORTFOLIO_FILE = "portfolio.json"


In [3]:
# Binance’te takip edeceğimiz 20 coin


TOP20 = [
    "BTCUSDT","ETHUSDT","BNBUSDT","XRPUSDT","ADAUSDT",
    "DOGEUSDT","SOLUSDT","TRXUSDT","DOTUSDT","MATICUSDT",
    "LTCUSDT","SHIBUSDT","AVAXUSDT","LINKUSDT","ATOMUSDT",
    "XMRUSDT","ETCUSDT","XLMUSDT","APTUSDT","ICPUSDT"
]


In [4]:
# Portföyü JSON dosyasına yaz/oku
# Kullanıcı coin eklediğinde dosyaya kaydedilir


def load_portfolio():
    """JSON dosyasından portföy oku"""
    if os.path.exists(PORTFOLIO_FILE):
        with open(PORTFOLIO_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_portfolio(pf):
    """Portföyü JSON dosyasına kaydet"""
    with open(PORTFOLIO_FILE, "w", encoding="utf-8") as f:
        json.dump(pf, f, indent=2)


In [5]:
# Adım 4: Binance API’den fiyat verilerini çek
# /api/v3/ticker/price endpoint → anlık fiyat döner


def get_prices_binance(symbols=TOP20):
    url = "https://api.binance.com/api/v3/ticker/price"
    r = requests.get(url, timeout=10)
    data = r.json()
    rows = []
    by_symbol = {}

    for d in data:
        if d["symbol"] in symbols:
            sym = d["symbol"].replace("USDT","")   # Örn: BTCUSDT → BTC
            price = float(d["price"])
            row = {"symbol": sym, "pair": d["symbol"], "price": price}
            rows.append(row)
            by_symbol[sym.upper()] = row

    return by_symbol, rows


In [6]:
# Adım 5: Fiyatları okunabilir formata çevir


def fmt_price(v):
    if v >= 1000: return f"{v:,.2f}"
    elif v >= 1: return f"{v:,.2f}"
    elif v >= 0.1: return f"{v:.4f}"
    elif v >= 0.01: return f"{v:.5f}"
    elif v >= 0.001: return f"{v:.6f}"
    else: return f"{v:.8f}"


In [7]:
# HTML şablonu
# - Binance Top 20 coin tablosu
# - Portföy tablosu (ekle/sil formları)
# - Auto-refresh varsayılan 10 sn

TEMPLATE = """
<!doctype html>
<html lang="tr" data-bs-theme="dark">
<head>
  <meta charset="utf-8">
  <title>Crypto Portfolio & Binance Top 20</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css">
  <style>.gain{color:#18c964}.loss{color:#ff6b6b}.mono{font-family:monospace;}</style>
  <script>
    //  Auto-refresh (varsayılan 10 sn)
    const r = parseInt(new URLSearchParams(location.search).get("refresh") || "10");
    if (r > 0) setTimeout(() => location.reload(), r * 1000);
  </script>
</head>
<body class="bg-dark text-light">
<div class="container py-4">

  <!-- Sayfa başlığı -->
  <div class="d-flex justify-content-between mb-3">
    <h1 class="h3"> Binance Top 20 & Portföy</h1>
    <div class="small text-secondary">Son güncelleme: {{ updated_at }}</div>
  </div>

  <!-- Binance Top 20 Coin Tablosu -->
  <div class="card mb-4">
    <div class="card-header d-flex justify-content-between align-items-center">
      <b>Binance USDT Çiftleri (Top 20)</b>
      <!-- Kullanıcı yenileme süresini değiştirebilir -->
      <form class="d-flex gap-2" method="get">
        <input type="number" min="0" step="1" name="refresh" value="{{ refresh }}"
               class="form-control form-control-sm" style="width:90px">
        <button class="btn btn-sm btn-outline-light">Yenile</button>
      </form>
    </div>
    <table class="table table-dark table-striped m-0">
      <thead><tr><th>#</th><th>Coin</th><th>Fiyat (USDT)</th><th>Portföye Ekle</th></tr></thead>
      <tbody>
      {% for coin in rows %}
        <tr>
          <td>{{ loop.index }}</td>
          <td>{{ coin.symbol }}</td>
          <td class="text-end mono">${{ fmt_price(coin.price) }}</td>
          <td>
            <!-- Portföye ekleme formu -->
            <form method="post" action="{{ url_for('add') }}" class="d-flex gap-2">
              <input type="hidden" name="symbol" value="{{ coin.symbol }}">
              <input type="number" step="any" min="0" name="amount" placeholder="miktar" required class="form-control form-control-sm" style="width:100px">
              <input type="number" step="any" min="0" name="buy_price" placeholder="maliyet ($)" required class="form-control form-control-sm" style="width:120px">
              <button class="btn btn-sm btn-primary">Ekle</button>
            </form>
          </td>
        </tr>
      {% endfor %}
      </tbody>
    </table>
  </div>

  <!-- Portföy Tablosu -->
  <div class="card">
    <div class="card-header">Portföyüm</div>
    <div class="card-body">
      {% if portfolio_items %}
      <table class="table table-dark table-striped">
        <thead><tr><th>Coin</th><th>Miktar</th><th>Alış ($)</th><th>Fiyat ($)</th><th>Değer ($)</th><th>K/Z</th><th></th></tr></thead>
        <tbody>
        {% for row in portfolio_items %}
          <tr>
            <td>{{ row.sym }}</td>
            <td>{{ row.amount }}</td>
            <td>{{ fmt_price(row.buy_price) }}</td>
            <td>{{ fmt_price(row.price) }}</td>
            <td>{{ fmt_price(row.value) }}</td>
            <td class="{% if row.pnl>=0 %}gain{% else %}loss{% endif %}">{{ fmt_price(row.pnl) }}</td>
            <td>
              <!-- Portföyden silme formu -->
              <form method="post" action="{{ url_for('remove') }}">
                <input type="hidden" name="symbol" value="{{ row.sym }}">
                <button class="btn btn-sm btn-danger">Sil</button>
              </form>
            </td>
          </tr>
        {% endfor %}
        </tbody>
      </table>
      {% else %}
        <p class="text-secondary">Portföy boş.</p>
      {% endif %}
    </div>
  </div>

</div>
</body>
</html>
"""


In [8]:
# Flask Routes
# - home(): Top 20 fiyatları + portföyü göster
# - add(): Portföye coin ekle
# - remove(): Portföyden coin sil


@app.route("/")
def home():
    refresh = int(request.args.get("refresh","10") or 10)
    by_symbol, rows = get_prices_binance()
    pf = load_portfolio()

    items = []
    for sym, pos in pf.items():
        coin = by_symbol.get(sym)
        if not coin: 
            continue
        price = coin["price"]
        amount = float(pos["amount"])
        buy_price = float(pos["buy_price"])
        value, cost = amount*price, amount*buy_price
        pnl = value-cost
        items.append({"sym":sym,"amount":amount,"buy_price":buy_price,
                      "price":price,"value":value,"pnl":pnl})
    return render_template_string(TEMPLATE,
        rows=rows, refresh=refresh,
        fmt_price=fmt_price, portfolio_items=items,
        updated_at=datetime.now().strftime("%H:%M:%S"))

@app.route("/add", methods=["POST"])
def add():
    sym = request.form.get("symbol","").upper().strip()
    amount = float(request.form.get("amount","0") or 0)
    buy_price = float(request.form.get("buy_price","0") or 0)
    if amount<=0 or buy_price<=0 or not sym: 
        return redirect(url_for("home"))
    pf = load_portfolio()
    pf[sym] = {"amount":amount,"buy_price":buy_price}
    save_portfolio(pf)
    return redirect(url_for("home"))

@app.route("/remove", methods=["POST"])
def remove():
    sym = request.form.get("symbol","").upper().strip()
    pf = load_portfolio()
    if sym in pf: 
        pf.pop(sym)
        save_portfolio(pf)
    return redirect(url_for("home"))


In [ ]:
# Flask uygulamasını çalıştır

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Oct/2025 23:45:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:45:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:45:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:45:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:45:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:46:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:46:15] "GET /?refresh=20 HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:46:28] "POST /add HTTP/1.1" 302 -
127.0.0.1 - - [02/Oct/2025 23:46:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:46:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:46:45] "GET /?refresh=10 HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:46:56] "GET /?refresh=10 HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:47:06] "GET /?refresh=10 HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:47:16] "GET /?refresh=10 HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2025 23:47:27] "GET